# Index and search single cell data with scfind

> We recommend setting the memory on the workspace advanced configuration settings to at least 32GB since building the scfind index can be memory-consuming.

This notebook shows how to do fast searches of large collections of single cell data with scfind.
scfind is an advanced single-cell analysis tool that facilitates fast search of biologically relevant marker genes in cell atlases. Utilizing an efficient compression strategy for sparse single-cell measurements, scfind builds a compact index to store the expression matrix, enabling fast queries at single-cell resolution. The central operation carried out by scfind is to identify the set of cells expressing user-specified gene combinations. This functionality supports multiple analytical applications, including marker gene identification and evaluation, in silico gating, and identifying both cell-type-specific and housekeeping genes. Furthermore, scfind incorporates a subquery optimization routine, ensuring that long and complex queries return biologically meaningful results.

In [ ]:
!pip install pybind11 numpy==1.24.4 pandas scipy==1.10.1 anndata==0.9.2 gensim==4.3.2 statsmodels tqdm python-Levenshtein smart_open h5py requests hubmap-template-helper
!pip install -i https://test.pypi.org/simple/ scfind==2.3

In [ ]:
import requests
import json
import warnings
import os
import math

import anndata as ad

import scfind

from hubmap_template_helper import compatibility as hth_comp

# Linked datasets
The following datasets were symlinked to the workspace when this template was added:

In [ ]:
# linked datasets
uuids = {{ uuids | safe }}

# accepted assay_display_names
accepted_assay_display_names = ["snRNAseq (SNAREseq2) [Salmon]"]

# search_api
search_api = 'https://search.api.hubmapconsortium.org/v3/portal/search'

The following checks if the datasets are compatible with this template.

In [ ]:
uuids = hth_comp.check_template_compatibility(uuids, accepted_assay_display_names=accepted_assay_display_names, search_api=search_api)

In [ ]:
uuids

# Groupings
This notebook shows how to create two groupings and perform scfind on both.

For this we need at least two datasets.

In [ ]:
if len(uuids) < 2: 
    warnings.warn("Not enough datasets to run this analysis")

As an example, we will split the datasets into two groupings based on when they were published. Of course this can be changed to something more biologically meaningful.

In [ ]:
hits = json.loads(
    requests.post(
        search_api,
        json={
            "size": 10000,  # To make sure the list is not truncted, set this high.
            "query": {"ids": {"values": uuids}},
        },
    ).text
)["hits"]["hits"]

list_ids_published = []
for hit in hits:
    list_ids_published.append([hit["_id"], hit["_source"]["published_timestamp"]])
list_ids_published.sort(key = lambda x: x[1])

half = math.ceil(len(list_ids_published)/2)

uuids_group_1 = [id[0] for id in list_ids_published[0:half]]
uuids_group_2 = [id[0] for id in list_ids_published[half:]]

print(f"Group 1: {uuids_group_1}")
print(f"Group 2: {uuids_group_2}")

# Reading in data and preprocessing, building scfind index
The initial step involves reading the anndata of the datasets of interest.

Building the scfind index requires one or more nonnegative matrices as input. In these matrices, columns represent genes, peaks, or other features that serve as query terms, while rows represent individual cells. While there are no strict normalization requirements, a sparse matrix structure is advantageous for efficient processing.

This example utilizes an annotated AnnData object loaded from _secondary_analysis.h5ad_, with cell annotations stored in AnnData.obs and feature names stored in AnnData.var. During index construction, a dataset_name parameter can be provided. This allows for differentiation of cell types from various datasets, potentially representing organ/tissue names, donor IDs, experimental conditions, or any other meaningful identifiers to distinguish cell groups from different datasets. In this example, we add the groupings based on the time of publishing as created above. Following this step, queries can be specified within the datasets of interest.

The index building process begins with the initialization of the scfind object and building index using a single AnnData object. Subsequently, any remaining AnnData objects are merged into the existing index. This approach conserves memory when multiple large datasets are loaded. 

> This might show certain errors about detected vectors of zeros. This is to be expected.

In [ ]:
## Build scfind index for datasets for both groups

cell_type_label = 'predicted_label'
feature_name = 'hugo_symbol'
layer_to_use = 'spliced'

# initializing scfind object
scfind_index = scfind.SCFind()

for i, uuid in enumerate(uuids):
    print(f"Building index for dataset {i}: {uuid}.")

    # check which group this belongs to
    
    group = "group-1" if uuid in uuids_group_1 else "group-2"  
    print(f"Dataset is part of group: {group}")
    
    # read in data and preprocess
    adata = ad.read_h5ad(os.path.join('datasets', uuid, 'secondary_analysis.h5ad'))
    valid_genes = adata.var[feature_name].dropna().drop_duplicates()
    adata = adata[:, list(valid_genes.index)]
    adata = ad.AnnData(X=adata.layers[layer_to_use], obs=adata.obs, var=adata.var)

    # create index
    # if this is the first dataset, we will add it to our intilized index
    if i == 0:
        scfind_index.buildCellTypeIndex(
            adata=adata, 
            dataset_name=group,
            feature_name=feature_name,
            cell_type_label=cell_type_label
        )

    # if this is not the first dataset, we will set it to a new scfind index, and merge it to our existing index
    else:
        scfind_index_new = scfind.SCFind()
        scfind_index_new.buildCellTypeIndex(
            adata=adata,
            dataset_name=group,
            feature_name=feature_name,
            cell_type_label=cell_type_label,
        )
        scfind_index.mergeDataset(scfind_index_new)

Index can be saved and reloaded for future use

In [ ]:
# save index 
scfind_index.saveObject("scfind_index.bin")

In [ ]:
# Reload index 
scfind_index = scfind.SCFind()
scfind_index.loadObject("scfind_index.bin")

Now with the scfind index we can check the basic information, e.g., genes, cell types, and datasets contained in scfind index.

In [ ]:
print(f"Number of genes in index: {len(scfind_index.scfindGenes)}")
print(scfind_index.scfindGenes[:10])

In [ ]:
print(f"Number of datasets: {len(scfind_index.datasets)}")
print(scfind_index.datasets)

In [ ]:
datasets = [d for ds in scfind_index.datasets_map.values() for d in ds]
print(f"Total number of cell in index: {scfind_index.index.getTotalCells(datasets)}")
print(f"Total number of cell types in index: {len(scfind_index.cellTypeNames())}")
print(f"{scfind_index.cellTypeNames()[0:6]}")

# Use scfind index for fast query

The central operation of scfind is to identify the set of cells that express a set of genes (query) specified by the user. With the basic operation scfind can be used to identify and evaluate marker genes, perform in silico gating and identify both cell-type-specific and housekeeping genes.

## Identify cell type markers with index.cellTypeMarkers
The index.cellTypeMarkers function identifies marker genes for cell types of interest compared to specified background cell types. For each gene, four key metrics are defined:
1. tp (true positive): Number of cells with non-zero expression values of a specific gene in the given cell types.
2. fp (false positive): Number of cells with non-zero expression values of the gene outside the given cell types.
3. fn (false negative): Number of cells with zero expression values of the gene in the given cell types.
4. tn (true negative): Number of cells with zero expression values outside the given cell types (not shown in results table).

Based on these metrics, scfind computes the following:
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)
F1 score = 2 * (precision * recall) / (precision + recall)

A higher F1 score indicates that the gene is more exclusively expressed in the cell types of interest. This measure provides a balanced assessment of a gene's specificity and sensitivity as a marker for the given cell types.

In [ ]:
ct = scfind_index.cellTypeNames()[0]
bg_cts = scfind_index.cellTypeNames()[0:5]
scfind_index.cellTypeMarkers(cell_types=ct, background_cell_types = bg_cts, top_k=10) 

If background cell types are not specified, all cell types in index would be used as background.

In [ ]:
scfind_index.cellTypeMarkers(cell_types=ct)

## Evaluate cell type markers with index.evaluateMarkers
The index.evaluateMarkers function assesses user-specified genes in designated cell type(s) of interest against background cell types. This evaluation employs the same metrics utilized in the index.cellTypeMarkers method, to evaluate the specified genes as markers for the cell types of interest.

Upon evaluation of gene expression in T cell populations, the T cell-associated markers CCR7, CD4, and CD27 exhibites higher F1 scores compared to the epithelial-associated markers KRT18, CDH1, and CLDN3.

In [ ]:
genes = ['KRT18', 'CDH1', 'CLDN3', 'CCR7', 'CD4', 'CD27']
ct = 'group-1.T cell'
scfind_index.evaluateMarkers(gene_list=genes, cell_types=ct, background_cell_types=bg_cts)

## Significant enrichment analysis of a list of genes by index.hyperQueryCellTypes
Scfind also supports hypergeometric test to identify the cell types that a query of interest is significantly enriched in. The output counts the number of cells expressing all genes in query (cell_hits) in each cell type and total number of cells in the cell type, as well as the p values by hypergeometric test and modified by multiple testing correction.

In [ ]:
# scfind identifies the B cells with marker genes successfully
genes = ['PLEK', 'PRKCB']
datasets = scfind_index.datasets[0]
scfind_index.hyperQueryCellTypes(gene_list=genes, datasets=datasets)

The hyperQueryCellTypes method supports in silico gating, allowing the use of operators to refine the query:

-gene: Exclude cells expressing "gene."
*gene: Include cells if either "gene" is expressed.
*-gene: Exclude cells if either "gene" is expressed.

For example, if we are interested in cell types that significantly express HLA-DRA and CD38, but not IGHA1, the input query would be: ['HLA-DRA', 'CD38', '-IGHA1'].

In [ ]:
genes = ['HLA-DRA', 'CD38', '-IGHA1']
datasets = scfind_index.datasets[0]
scfind_index.hyperQueryCellTypes(gene_list=genes, datasets=datasets)

## Query optimization for long queries (>5) with index.markerGenes
For queries involving even a moderate number (>5) of terms for sparsely sequenced datasets, it is likely that an empty set of cells will be returned. To ensure that meaningful results are returned without requiring the user to manually modify the query, scfind features a query optimization routine to identify subqueries that are guaranteed to return nonempty sets of cells.

The output of index.markerGenes consists of subqueries, and a score measuring the importance of the subqueries to given datasets, as well as the number of cells with non-zero values of each subquery.

In [ ]:
genes = scfind_index.scfindGenes[0:10]
scfind_index.markerGenes(gene_list=genes)

### Identification of housekeeping genes by index.findHouseKeepingGenes
Of particular interest are those genes expressed across most or all cell types, also known as housekeeping genes. scfind supports the identification of housekeeping genes for given cell types.

In [ ]:
# scfind allows to identify the cell-type-specific genes and housekeeping genes across cell types.
cts = scfind_index.cellTypeNames()[0:3]
scfind_index.findHouseKeepingGenes(cell_types=cts, )

### Retrieve of gene expression data by index.getCellTypeExpression
scfind also approximates the original non-zero expression values by quantized expressione values. Given a cell type of interest, The quantized expression data would be returned in AnnData object.

In [ ]:
ct = scfind_index.cellTypeNames()[0]
adata = scfind_index.getCellTypeExpression(cell_type=ct)
adata